In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from pathlib import Path
import os

## Run simulation

In [ ]:
from symmetry_breaking.models.NL_field_1D import NodalLeftyField1D
from symmetry_breaking.models.trackers import NodalROITracker
from symmetry_breaking.models.sweep import run_simulation_1D, make_1d_grid
from pde import ProgressTracker, PlotTracker

# sim_id = "c4bdd83d"

# hyperparams
dx = 5
L = 3500
T = 10 * 60 * 60
dt = 0.5 * dx ** 2 / 60 / 1.25 # Stability condition for diffusio

param_dict = {
         'K_NL': 138.9495494373,
         'K_A': 193.0697728883,
         'K_I': 19.3069772888,
         'mu_L': 0.0002275846, #5e-5,
         'N_amp': 0, #100, #1,
         'N_sigma': 31.6227766017,
         'sigma_N': 10.0,
         'alpha' : 0.3,
         'sigma_L': 10.0,
         'D_N': 1.85,
         'D_L': 15,
         'n': 2,
         'm': 1,
         'p': 2,
         'q': 2,
         'stoch_to_N':True, 
         'rate_N':1/(3500*60)/1, 
         'amp_median_N':500, 
         'amp_sigma_N':.1,
         "sigma_x":31,
         # "L_value": 100,
         'stoch_N_mode':"direct"}   # not "impulse"}

static_params = {
                }

sim_config = {
                "dx": dx,
                "L": L,
                "T": T,
                "dt": dt,
                "model_class": NodalLeftyField1D,
                "tracker_class": NodalROITracker
            }

sim_inputs = (param_dict | static_params, sim_config, "") 

In [ ]:
param_dict, sim_config, output_dir = sim_inputs

# Unpack sim config
dx = sim_config["dx"]
L = sim_config["L"]
T = sim_config["T"]
dt = sim_config["dt"]
interval=300
model_class = sim_config["model_class"]
tracker_class = sim_config["tracker_class"]
# interval = sim_config.get("interval", 1000)

# --- Setup grid and model ---
grid = make_1d_grid(length=L, dx=dx)
model = model_class(**param_dict)
state = model.get_state(grid)

# roi = NodalROITracker(grid, interval=interval,
#                       save_profiles=False,
#                       store_every=300,      # set e.g. 100 if you want snapshots too
#                       downsample=None,          # optional
#                       dtype=np.float32)

# --- Build a tracker collection ---
# Progress bar updates every 'interval' steps
progress = ProgressTracker(interval=interval)

# Live plots (you can adjust scale, cmap, etc.)
plots = PlotTracker(interval=interval, plot_args={"figsize": (8, 4)}, 
                    show=True, tight_layout=True)

# If you want both at once, wrap them in a list
trackers = [plots, progress, tracker_class(grid, interval=interval)]

# --- Run simulation ---
state = model.solve(state, t_range=T, dt=dt, tracker=trackers)

# --- Collect results from your custom tracker ---
result = {
    **param_dict,
    **trackers[-1].get_metrics(),  # last tracker is your custom one
}

In [ ]:
profiles = trackers[-1].get_profiles()
x = profiles["x"] - 1750
time_vec = profiles["times"]
N = profiles["Activator"]      # numpy array
L = profiles["Repressor"]      # numpy array
rho = profiles.get("rho") 

In [ ]:
from tqdm import tqdm

frame_path = fig_path / f"{sim_id}_frames"
os.makedirs(frame_path, exist_ok=True)

for t, time in enumerate(tqdm(time_vec)):
    fig = go.Figure()
    fig.add_traces(go.Scatter(
        x=x, 
        y=N[t, :] , 
        mode="lines", 
        line=dict(width=3, color="#8da0cb"),  # Set2 blue
        fill="tozeroy",
        fillcolor="rgba(141, 160, 203, 0.6)",  # semi-transparent blue
        name="Nodal"
    ))
    fig.add_traces(go.Scatter(
        x=x, 
        y=L[t, :], 
        mode="lines", 
        line=dict(width=3, color="#fc8d62"),  # Set2 red
        fill="tozeroy",
        fillcolor="rgba(252, 141, 98, 0.3)",  # semi-transparent red
        name="Lefty"
    ))
    
    fig = format_2d_plotly(fig, axis_labels=["position (microns)", "concentration (nmol per micron squared)"], 
                           font_size=18)
    fig.update_layout(title=f"Nodal and Lefty concentrations ({np.round(time/3600,2)} hours)")
    fig.update_xaxes(range=[-1500, 1500])
    fig.update_yaxes(range=[0, 20000])
    
    # fig.show()
    fig.write_image(frame_path / f"frame_{t:05}.png")

fig.show()

### Re-run the exact same simulation but with no link between Lefty and Nodal 

In [ ]:
# hyperparams
dx = 5
L = 3500
T = 10 * 60 * 60
dt = 0.5 * dx ** 2 / 60 / 1.25 # Stability condition for diffusio

param_dict = {
         'K_NL': 138.9495494373,
         'K_A': 193.0697728883,
         # 'K_R': 19.3069772888,
         'K_I': 1e10, #19.3069772888,
         'mu_L': 0.0002275846,
         'N_amp': 517.9474679231,
         'N_sigma': 31.6227766017,
         'sigma_N': 10.0,
         'sigma_L': 10.0,
         'D0_N': 1.85,
         'D0_L': 15.0,
         'alpha_L': 0,
         'alpha_N': 0,
         'tau_rho': 3600,
         'n': 2,
         'm': 1,
         'p': 2,
         'q': 2}

static_params = {
                  "sigma_N": 10.0,  # Nodal auto-activation
                  "no_density_dependence": True,
                  "alpha_L": 0,
                  "alpha_N": 0,
                  "tau_rho": 3600,
                }

sim_config = {
                "dx": dx,
                "L": L,
                "T": T,
                "dt": dt,
                "model_class": NodalLeftyNeutralization1D,
                "tracker_class": NodalROITracker,
                "interval": 1000,
            }

sim_inputs = (param_dict | static_params, sim_config, "") 

In [ ]:
25**2

In [ ]:
param_dict, sim_config, output_dir = sim_inputs

# Unpack sim config
dx = sim_config["dx"]
L = sim_config["L"]
T = sim_config["T"]
dt = sim_config["dt"]
interval=300
model_class = sim_config["model_class"]
tracker_class = sim_config["tracker_class"]
# interval = sim_config.get("interval", 1000)

# --- Setup grid and model ---
grid = make_1d_grid(length=L, dx=dx)
model = model_class(**param_dict)
state = model.get_state(grid)

# roi = NodalROITracker(grid, interval=interval,
#                       save_profiles=False,
#                       store_every=300,      # set e.g. 100 if you want snapshots too
#                       downsample=None,          # optional
#                       dtype=np.float32)

# --- Build a tracker collection ---
# Progress bar updates every 'interval' steps
progress = ProgressTracker(interval=interval)

# Live plots (you can adjust scale, cmap, etc.)
# plots = PlotTracker(interval=interval, plot_args={"figsize": (6, 4)}, 
#                     show=True, tight_layout=True)

# If you want both at once, wrap them in a list
trackers = [progress, tracker_class(grid, interval=interval)]

# --- Run simulation ---
state = model.solve(state, t_range=T, dt=dt, tracker=trackers)

# --- Collect results from your custom tracker ---
result = {
    **param_dict,
    **trackers[-1].get_metrics(),  # last tracker is your custom one
}

In [ ]:
profiles = trackers[-1].get_profiles()
x = profiles["x"]- 1750
time_vec = profiles["times"]
N = profiles["Activator"]      # numpy array
L = profiles["Repressor"]      # numpy array
rho = profiles.get("rho") 

In [ ]:
frame_path = fig_path / f"{sim_id}_frames_no_LN"
os.makedirs(frame_path, exist_ok=True)

for t, time in enumerate(tqdm(time_vec)):
    fig = go.Figure()
    fig.add_traces(go.Scatter(
        x=x, 
        y=N[t, :] , 
        mode="lines", 
        line=dict(width=3, color="#8da0cb"),  # Set2 blue
        fill="tozeroy",
        fillcolor="rgba(141, 160, 203, 0.8)",  # semi-transparent blue
        name="Nodal"
    ))
    fig.add_traces(go.Scatter(
        x=x, 
        y=L[t, :], 
        mode="lines", 
        line=dict(width=3, color="#fc8d62"),  # Set2 red
        fill="tozeroy",
        fillcolor="rgba(252, 141, 98, 0.3)",  # semi-transparent red
        name="Lefty"
    ))
    
    fig = format_2d_plotly(fig, axis_labels=["position (microns)", "concentration (nmol per micron squared)"], 
                           font_size=18)
    fig.update_layout(title=f"Nodal and Lefty concentrations ({np.round(time/3600,2)} hours)")
    fig.update_xaxes(range=[-1500, 1500])
    fig.update_yaxes(range=[0, 50000])
    
    # fig.show()
    fig.write_image(frame_path / f"frame_{t:05}.png")

fig.show()

### Add random field